# Лабораторная работа №3
# Подготовка обучающей и тестовой выборки, кросс-валидация и подбор гиперпараметров на примере метода ближайших соседей

Выполнил: **Пакало А. С., РТ5-61Б**

## Задание
Выберите набор данных (датасет) для решения задачи классификации или
регрессии.
- С использованием метода train_test_split разделите выборку на обучающую и
  тестовую.
- Обучите модель ближайших соседей для произвольно заданного гиперпараметра
K.
  Оцените качество модели с помощью подходящих для задачи метрик.
- Произведите подбор гиперпараметра K с использованием GridSearchCV и/или
  RandomizedSearchCV и кросс-валидации, оцените качество оптимальной модели.
  Желательно использование нескольких стратегий кросс-валидации.
- Сравните метрики качества исходной и оптимальной моделей.

## Текстовое описание набора данных
Для обучения по методу K ближайших соседей (KNN) был выбран датасет с
классификацией типа звёзд c ресурса kaggle (Star Type Classification / NASA).
 
В данном наборе данных присутствуют следующие столбцы:
* Temperature — температура звезды в Кельвинах;
* L (Luminosity) — светимость звезды в солнечных светимостях;
* R (Radius) — радиус звезды в радиусах солнца;
* A_M (Absolute Magnitude) — абсолютная звёздная величина;
* Color — цвет света звезды;
* Spectral_Class — спектральный класс звезды;
* Type — тип звезды. Является целевым признаком и уже закодирован:
  - Красный карлик — 0;
  - Коричневый карлик — 1;
  - Белый карлик — 2;
  - Звезда из главной последовательности — 3;
  - Супергигант — 4;
  - Гипергигант — 5.

Так как данных очень много, перед тем как приступить к анализу, проведем обзор данных и, возможно, потребуется их предобработка, чтобы датасет стал более удобным и пригодным к проведению исследования.

Таким образом исследование пройдет в 7 этапов:
- загрузка данных,
- проведение разведочного анализа данных и предобработка данных,
- разделение на обучающую и тестовую выборку,
- выбор метрики,
- обучение модели,
- подбор гиперпараметра,
- сравнение значений метрики.

## Импортирование необходимых библиотек, подготовка окружения

In [64]:
# Основные библиотеки.
import numpy as np
import pandas as pd

# Визуализция.
import matplotlib.pyplot as plt
import seaborn as sns

# Для матрицы взаимодействий.
from scipy import sparse
# Для разбития выборки.
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from DataFrameOneHotEncoder import DataFrameOneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Отрисовка статуса выполнения.
from tqdm.notebook import tqdm

# Типизация.
from typing import List

# Вывод данных.
from IPython.display import display, Markdown
def printmd(message):
    display(Markdown(message))


# Конфигурация визуализации.
%matplotlib inline
sns.set(style='ticks')

## Загрузка данных
### Считываем данные из .csv
Загрузим файлы датасета в помощью библиотеки Pandas.

Не смотря на то, что файлы имеют расширение txt они представляют собой данные
в формате [CSV](https://ru.wikipedia.org/wiki/CSV). Часто в файлах такого
формата в качестве разделителей используются символы ",", ";" или табуляция.
Поэтому вызывая метод read_csv всегда стоит явно указывать разделитель данных
с помощью параметра sep. Чтобы узнать какой разделитель используется в файле
его рекомендуется предварительно посмотреть в любом текстовом редакторе.

In [65]:
stars = pd.read_csv('data/Stars.csv')

## Проведение разведочного анализа данных. Построение графиков, необходимых для понимания структуры данных. Анализ и предобработка данных.

Размеры датасета: (строки, колонки).

In [66]:
stars.shape

(240, 7)

Общий вид данных таблицы:

In [67]:
stars.head()

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0


Список колонок:

In [68]:
stars.columns

Index(['Temperature', 'L', 'R', 'A_M', 'Color', 'Spectral_Class', 'Type'], dtype='object')

Список колонок с типами данных:

In [69]:
stars.dtypes

Temperature         int64
L                 float64
R                 float64
A_M               float64
Color              object
Spectral_Class     object
Type                int64
dtype: object

Как видно, все данные приведены к адекватному типу данных.

## Предобработка данных

In [70]:
# Извлекаем целевой признак.
TARGET_COL_NAMES = ['Type']
star_types = stars[TARGET_COL_NAMES]

star_features = stars.drop(columns=TARGET_COL_NAMES)
display(star_types, star_features)

,Type
0,0
1,0
2,0
3,0
4,0
...,...
235,5
236,5
237,5
238,5


,Temperature,L,R,A_M,Color,Spectral_Class
0,3068,0.002400,0.1700,16.12,Red,M
1,3042,0.000500,0.1542,16.60,Red,M
2,2600,0.000300,0.1020,18.70,Red,M
3,2800,0.000200,0.1600,16.65,Red,M
4,1939,0.000138,0.1030,20.06,Red,M
...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,Blue,O
236,30839,834042.000000,1194.0000,-10.63,Blue,O
237,8829,537493.000000,1423.0000,-10.73,White,A
238,9235,404940.000000,1112.0000,-11.23,White,A


In [71]:
# Перед использованием модели закодируем категориальные признаки с помощью
# one-hot encoding, где каждое уникальное значение признака становится новым
# признаком. Это позволяет избежать фиктивного отношения порядка.

In [72]:
categorical_pipeline = Pipeline([
    ( 'one-hot', DataFrameOneHotEncoder(handle_unknown='ignore') )
])
CATEGORICAL_COL_NAMES = ['Color', 'Spectral_Class']
# Returns tuple: (2d-array with columns?, shape).
caterogical_star_features = categorical_pipeline.fit_transform(stars[CATEGORICAL_COL_NAMES]),
caterogical_star_features = pd.DataFrame(caterogical_star_features[0])

Нам также потребуется масштабировать данные для адекватной работы модели.

In [73]:
numerical_pipeline = Pipeline([
    ( 'scaler', StandardScaler() )
])


numerical_star_features = star_features.drop(columns=CATEGORICAL_COL_NAMES)
numerical_star_features_transformed = numerical_pipeline.fit_transform(numerical_star_features)
# Массив переводим обратно в датафрейм.
numerical_star_features_transformed = pd.DataFrame(numerical_star_features,
                                                   columns=numerical_star_features.columns)
numerical_star_features_transformed

,Temperature,L,R,A_M
0,3068,0.002400,0.1700,16.12
1,3042,0.000500,0.1542,16.60
2,2600,0.000300,0.1020,18.70
3,2800,0.000200,0.1600,16.65
4,1939,0.000138,0.1030,20.06
...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93
236,30839,834042.000000,1194.0000,-10.63
237,8829,537493.000000,1423.0000,-10.73
238,9235,404940.000000,1112.0000,-11.23


In [74]:
NUMERICAL_COL_NAMES = list(filter(lambda feature:
        feature not in
        CATEGORICAL_COL_NAMES,
    star_features.columns))

preprocessor = ColumnTransformer([
    ( 'numerical', numerical_pipeline, NUMERICAL_COL_NAMES ),
    ( 'categorical', categorical_pipeline, CATEGORICAL_COL_NAMES)
])

star_features_preprocessed = preprocessor.fit_transform(star_features)

## С использованием метода train_test_split разделите выборку на обучающую и тестовую.

In [75]:

star_features_train: pd.DataFrame
star_features_test: pd.DataFrame
star_types_train: pd.Series
star_types_test: pd.Series

# Параметр random_state позволяет задавать базовое значение для генератора
# случайных чисел. Это делает разбиение неслучайным. Если задается параметр
# random_state то результаты разбиения будут одинаковыми при различных
# запусках. На практике этот параметр удобно использовать для создания
# "устойчивых" учебных примеров, которые выдают одинаковый результат при
# различных запусках.
RANDOM_STATE_SEED = 1

star_features_train, star_features_test, star_types_train, star_types_test = train_test_split(
    star_features, star_types, random_state=RANDOM_STATE_SEED)

Общий вид обучающей выборки:

In [76]:
pd.DataFrame(star_features_preprocessed)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-0.779382,-0.598624,-0.459210,1.116745,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.782110,-0.598624,-0.459241,1.162414,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.828477,-0.598624,-0.459342,1.362213,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.807496,-0.598624,-0.459229,1.167171,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.897819,-0.598624,-0.459340,1.491607,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2.983743,1.494720,2.167974,-1.361718,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
236,2.133913,4.059319,1.854068,-1.428317,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
237,-0.175029,2.403157,2.297800,-1.437832,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
238,-0.132438,1.662878,1.695177,-1.485403,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
display(star_features_train.head(), star_types_train.head())

,Temperature,L,R,A_M,Color,Spectral_Class
116,4015,282000.0000,1534.0000,-11.39,Red,K
200,16790,0.0014,0.0121,12.87,Blue,B
47,3574,200000.0000,89.0000,-5.24,Red,M
179,24490,248490.0000,1134.5000,-8.24,Blue-white,B
231,38234,272830.0000,1356.0000,-9.29,Blue,O


,Type
116,5
200,2
47,4
179,5
231,5


Общий вид тестовой выборки:

In [78]:
display(star_features_test.head(), star_types_test.head())

,Temperature,L,R,A_M,Color,Spectral_Class
228,23095,347820.00000,86.00000,-5.905,Blue,O
194,3523,0.00540,0.31900,12.430,Red,M
88,13720,0.00018,0.00892,12.970,white,F
95,11250,672.00000,6.98000,-2.300,Blue-white,A
214,34190,198200.00000,6.39000,-4.570,Blue,O


,Type
228,4
194,1
88,2
95,3
214,3


## Обучение и оценка модели ближайших соседей для произвольно заданного гиперпараметра
В классической модели ближайших соседей гиперпараметром является количество
соседей. Зададим его в константе N_NEIGHBORS.

In [82]:
from sklearn.metrics import mean_absolute_error

N_NEIGHBORS = 5
# В KNN Наиболее часто используется Евклидова метрика, поэтому для определения веса
#   соседей выберем параметр 'distance'.
knn_classifier = KNeighborsClassifier(n_neighbors=N_NEIGHBORS, weights='distance')

knn_pipeline = Pipeline([
    ( 'preprocess', preprocessor ),
    ( 'model', knn_classifier ),
])

star_types_predicted = knn_pipeline.fit(star_features_train,
                   star_types_train.values.ravel()).predict(star_features_test)
mean_absolute_error(star_types_test, star_types_predicted)

0.016666666666666666

## Произведите подбор гиперпараметра K с использованием GridSearchCV и/или RandomizedSearchCV и кросс-валидации, оцените качество оптимальной модели.
Желательно использование нескольких стратегий кросс-валидации.

In [91]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import SCORERS

params = { 'model__n_neighbors': np.arange(1, 30, 1) }

display(sorted(SCORERS.keys()))
grid_search = GridSearchCV(knn_pipeline, params, scoring='neg_mean_absolute_error')

grid_search.fit(star_features_train, star_types_train.values.ravel())
display(grid_search.best_score_, grid_search.best_params_)

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

-0.02222222222222222

{'model__n_neighbors': 1}

## Сравните метрики качества исходной и оптимальной моделей.